In [2]:
import geopandas
import ee
import geemap.foliumap as geemap
import geemap as gee

import numpy as np
import matplotlib.pyplot as plt
import folium
import requests
import rasterio
from rasterio.plot import show

In [9]:
def extract_pixel_value(img, region,bandName, scale):
  """Extract pixel value from an image for a given region"""
  value = img.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=region,
      scale=scale,
      maxPixels=1e9
  ).get(bandName)
  return value


# ##### Define a method for displaying Earth Engine image tiles to folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    layer = folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    )
    layer.add_to(self)
    return layer

In [47]:
def extract_gee_data(spec_file_path: str, data):
    # découper chaque géométrie en sous géométrie ayant une valeur unique fournie par GEE
    gfc = ee.Image("UMD/hansen/global_forest_change_2021_v1_9")

    output = []
    for input_zone in data:
        image = gfc.clip(input_zone)
        bands_list = image.getInfo()['bands']
        band = bands_list[0]
        width = band['dimensions'][0]
        height = band['dimensions'][1]
        treecover = image.select(band['id'])
        scale = image.projection().nominalScale().getInfo()

        reduction = treecover.reduceToVectors(
            geometry=None,
            crs=image.projection(),
            scale=scale,
            geometryType='polygon',
            eightConnected=True,
            labelProperty='zone',
            maxPixels=1e15,
            tileScale=16
        )
        output.append(input_zone)

    return output

    # OU

    # donner min, max, mean pour chaque géométrie donnée en entrée


ee.Initialize()
ma_zone = ee.Geometry.Polygon([[[166.387779807,-22.2276038],[166.387779807,-22.039679148],[166.50,-21.99],[166.606135574,-22.039679148],[166.606135574,-22.2276038]]])
result = extract_gee_data('', [ma_zone])
folium.Map.add_ee_layer = add_ee_layer
map = folium.Map(location=[(-22.2276038+-21.99)/2,(166.606135574+166.387779807)/2], zoom_start=12)
folium.GeoJson(ma_zone.toGeoJSON()).add_to(map)
for zone in result:
    folium.GeoJson(zone.toGeoJSON()).add_to(map)
display(map)

In [ ]:
gfc = ee.Image("UMD/hansen/global_forest_change_2021_v1_9")
image = gfc.clip(ma_zone)
bands_list = image.getInfo()['bands']
band = bands_list[0]
width = band['dimensions'][0]
height = band['dimensions'][1]
treecover = image.select(band['id'])
scale = image.projection().nominalScale().getInfo()

# https://developers.google.com/earth-engine/guides/reducers_reduce_to_vectors
vectors = treecover.reduceToVectors(
    geometry=None,
    crs=image.projection(),
    scale=scale,
    geometryType='polygon',
    eightConnected=True,
    labelProperty='zone',
    maxPixels=1e15,
    tileScale=16
).geometry()
result = ee.FeatureCollection([ee.Feature(vectors)])
map.add_ee_layer(result, {'min': 0.0, 'max': 100.0, 'palette': ['ffffff', 'ffbbbb', '0000ff']}, "toto")
display(map)

In [82]:
zones = treecover.gt(30).add(treecover.gt(60))
# zones = zones.updateMask(zones.neq(0))
vectors = zones.addBands(treecover).reduceToVectors(
    geometry=None,
    crs=image.projection(),
    scale=scale,
    geometryType='polygon',
    eightConnected=False,
    labelProperty='zone',
    maxPixels=1e15,
    tileScale=16
)
map.add_ee_layer(zones, {'min': 0, 'max': 2, 'palette': ['ffffff', '449152', '01330a']}, 'treecover')

# Make a display image for the vectors, add it to the map.
# display = ee.Image(0).updateMask(0).paint(vectors, '000000', 3)
# map.add_ee_layer(ee.Image(0).updateMask(0).paint(vectors, '000000', 3), {'palette': '000000'}, 'vectors')
display(map)

In [234]:
# import des spécifications
import yaml
from yaml.loader import SafeLoader
import geopandas as gpd
from shapely.geometry import Polygon
import json

spec_file_path = './input_spec_example.yml'

with open(spec_file_path) as f:
    specs = yaml.load(f, Loader=SafeLoader)

# création du fond de carte
map = folium.Map(location=[(-22.2276038+-21.99)/2,(166.606135574+166.387779807)/2], zoom_start=12)

# création de l'input : liste de geodataframe
ma_zone_ee = ee.Geometry.Polygon([[[166.387779807,-22.2276038],[166.387779807,-22.039679148],[166.50,-21.99],[166.606135574,-22.039679148],[166.606135574,-22.2276038]]])
coords = [(166.387779807,-22.2276038),(166.387779807,-22.039679148),(166.50,-21.99),(166.606135574,-22.039679148),(166.606135574,-22.2276038)]
ma_zone = Polygon(coords)
gdf = gpd.GeoDataFrame(crs=specs['epsg'], geometry=[ma_zone])
input = gdf

# affichage de la zone d'entrée sur la carte
folium.GeoJson(gdf.to_json()).add_to(map)

for geometry in input['geometry']:
    xx, yy = geometry.exterior.coords.xy
    coords = np.dstack((xx,yy)).tolist()
    print(coords)
    ee_zone = ee.Geometry.Polygon(coords)
    image_zonee = ee.Image(specs['confRaster']['uri_image']).select(specs['keepList']).clip(ee_zone)
    vectors = image_zonee.addBands(srcImg=image_zonee).reduceToVectors(
        geometry=None,
        crs=specs['epsg'],
        scale=scale,
        geometryType='polygon',
        eightConnected=False,
        labelProperty='zone',
        maxPixels=1e15,
        tileScale=16,
        reducer=ee.Reducer.minMax()
    )
    print(vectors.toList(vectors.size()).get(5059).getInfo())
    mapid = vectors.getMapId()
    folium.TileLayer(
        tiles=mapid['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        overlay=True,
        name='border',
      ).add_to(map)

display(map)

[[[166.387779807, -22.2276038], [166.387779807, -22.039679148], [166.5, -21.99], [166.606135574, -22.039679148], [166.606135574, -22.2276038], [166.387779807, -22.2276038]]]
{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[166.393, -22.116], [166.393, -22.11625], [166.39275, -22.11625], [166.39275, -22.11675], [166.39325, -22.11675], [166.39325, -22.117], [166.39350000000002, -22.117], [166.39350000000002, -22.11675], [166.39375, -22.11675], [166.39375, -22.116500000000002], [166.39325, -22.116500000000002], [166.39325, -22.11625], [166.39350000000002, -22.11625], [166.39350000000002, -22.116], [166.393, -22.116]]]}, 'id': '+665573-88465', 'properties': {'max': 37, 'min': 37, 'zone': 37}}


In [ ]:
# reducer=ee.Reducer.min()
vectorList = vectors.toList(vectors.size())
vectorList.get(0).getInfo()
mapid = vectors.getMapId()
map = folium.Map(location=[(-22.2276038+-21.99)/2,(166.606135574+166.387779807)/2], zoom_start=12)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='border',
  ).add_to(map)
display(map)